# Imports & Settings

In [1]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

In [2]:
%load_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import numpy as np
import qgrid
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, load_csv, write_xlsx, load_xlsx, set_project_dir
from pa_lib.data import (
    calc_col_partitioned,
    clean_up_categoricals,
    unfactorize,
    flatten,
    replace_col,
    cond_col,
    desc_col,
    unfactorize,
    as_dtype,
    flatten_multi_index_cols,
)
from pa_lib.util import obj_size, cap_words, normalize_rows, clear_row_max
from pa_lib.log import time_log, info
from pa_lib.vis import dive

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 200)

# Load EK data

In [4]:
set_project_dir('vkprog')

data_files('*ek*')

,size,mtime
name,,
bd_cluster_ek_info.feather,4.2 MB,04.09.19 11:28:49


In [5]:
ek_info = load_bin('bd_cluster_ek_info.feather')

2019-09-04 11:29:03 [INFO] Reading from file C:\Users\kpf\data\vkprog\bd_cluster_ek_info.feather
2019-09-04 11:29:03 [INFO] Finished loading binary file in 0.05s (0.05s CPU)


# Prepare PLZ mapping data

In [7]:
set_project_dir('Raumgliederung')

data_files('*.xlsx')

,size,mtime
name,,
Raumgliederungen.xlsx,112.0 KB,20.08.19 14:57:46
do-t-09.02-gwr-37.xlsx,414.7 KB,20.08.19 17:00:54


In [8]:
plz = load_xlsx("do-t-09.02-gwr-37.xlsx", sheet_name="PLZ4").rename(
    columns={"PLZ4": "PLZ", "%_IN_GDE": "PRC", "KTKZ": "KANTON", "GDENAMK": "NAME"}
)

2019-09-04 11:30:27 [INFO] Reading from file C:\Users\kpf\data\Raumgliederung\do-t-09.02-gwr-37.xlsx
2019-09-04 11:30:28 [INFO] Finished loading xlsx file in 0.55s (0.55s CPU)


In [9]:
plz.head(10)

,PLZ,PRC,KANTON,GDENR,NAME
0,8914,100.00,ZH,1,Aeugst am Albis
1,8909,14.34,ZH,2,Affoltern am Albis
2,8910,85.66,ZH,2,Affoltern am Albis
3,8906,100.00,ZH,3,Bonstetten
4,6340,0.77,ZH,4,Hausen am Albis
5,8915,72.06,ZH,4,Hausen am Albis
6,8925,27.17,ZH,4,Hausen am Albis
7,8908,100.00,ZH,5,Hedingen
8,8926,100.00,ZH,6,Kappel am Albis
9,8934,100.00,ZH,7,Knonau


### PLZ are not uniquely mapped to towns, not even cantons!

In [10]:
plz.PLZ.value_counts().head(10)

1148    8
3053    7
9107    7
2345    7
6110    7
8500    6
3800    6
9427    6
2720    6
2523    6
Name: PLZ, dtype: int64

In [11]:
def collect(s, sep=","):
    return sep.join(map(str, s[s.notna()].unique()))


plz.groupby("PLZ").agg({"KANTON": ["nunique", collect]}).sort_values(
    ("KANTON", "nunique"), ascending=False
).head(10)

KANTON          
     nunique   collect
PLZ                   
9107       3  AR,AI,SG
2814       3  SO,BL,JU
6390       3  UR,OW,NW
2827       3  BE,SO,JU
9428       3  AR,AI,SG
9427       3  AR,AI,SG
6010       3  LU,OW,NW
9450       3  AR,AI,SG
3254       2     BE,SO
9100       2     AR,SG

### Find best matches for town and canton by maximising PRC per PLZ

In [13]:
def main_features(df):
    return df.loc[df.PRC.idxmax(), ['GDENR', 'NAME', 'KANTON']]

plz_unique = plz.groupby('PLZ').apply(main_features)

# Load data on economic regions

In [14]:
reg = load_xlsx("Raumgliederungen.xlsx", sheet_name="Daten", skiprows=0, header=1)

# drop unused
reg.drop(0, axis="index", inplace=True)
reg.drop(["Bezirks-nummer", "Kantons-nummer"], axis="columns", inplace=True)

# rename columns
reg.rename(
    columns={
        "BFS Gde-nummer": "GDENR",
        "Gemeindename": "NAME",
        "Kanton": "KANTON",
        "Bezirksname": "BEZIRK",
        "Arbeitsmarktgrossregionen 2018": "GROSSREGION_ID",
        "Arbeitsmarktregionen 2018": "REGION_ID",
    },
    inplace=True,
)

# fix data types
reg = reg.astype({"GDENR": "int64", "GROSSREGION_ID": "int64", "REGION_ID": "int64"})

2019-09-04 11:32:17 [INFO] Reading from file C:\Users\kpf\data\Raumgliederung\Raumgliederungen.xlsx
2019-09-04 11:32:17 [INFO] Finished loading xlsx file in 0.19s (0.19s CPU)


In [15]:
reg.head()

,GDENR,NAME,KANTON,BEZIRK,GROSSREGION_ID,REGION_ID
1,1,Aeugst am Albis,ZH,Affoltern,12,12031
2,2,Affoltern am Albis,ZH,Affoltern,12,12031
3,3,Bonstetten,ZH,Affoltern,12,12034
4,4,Hausen am Albis,ZH,Affoltern,11,11060
5,5,Hedingen,ZH,Affoltern,12,12031


### Grossregionen

In [16]:
reg_grossreg = load_xlsx(
    "Raumgliederungen.xlsx",
    sheet_name="CH1+CL_GBAE2018+1.0",
    skiprows=0,
    header=1,
    index_col=0,
)

2019-09-04 11:32:25 [INFO] Reading from file C:\Users\kpf\data\Raumgliederung\Raumgliederungen.xlsx
2019-09-04 11:32:25 [INFO] Finished loading xlsx file in 0.17s (0.16s CPU)


In [17]:
reg_grossreg.head()

,Label
Code,
1,Region Genf
2,Region Lausanne
3,Region Neuenburg
4,Region Freiburg
5,Region Biel–Jura


### Regionen

In [18]:
reg_reg = load_xlsx(
    "Raumgliederungen.xlsx",
    sheet_name="CH1+CL_BAE2018+1.0",
    skiprows=0,
    header=1,
    index_col=0,
)

2019-09-04 11:32:30 [INFO] Reading from file C:\Users\kpf\data\Raumgliederung\Raumgliederungen.xlsx
2019-09-04 11:32:30 [INFO] Finished loading xlsx file in 0.18s (0.19s CPU)


In [19]:
reg_reg.head()

,Label
Code,
1011,Vernier–Lancy
1012,Genève
1013,Le Grand-Saconnex
1014,Nyon
1015,Thônex–Chêne-Bougeries


### Merge Grossregionen and Regionen

In [20]:
reg = (
    reg.assign(
        REGION=reg_reg.loc[reg.REGION_ID].values,
        GROSSREGION=reg_grossreg.loc[reg.GROSSREGION_ID].values,
    )
    .drop(["GROSSREGION_ID", "REGION_ID"], axis="columns")
    .set_index("GDENR")
)

In [21]:
reg.head()

,NAME,KANTON,BEZIRK,REGION,GROSSREGION
GDENR,,,,,
1,Aeugst am Albis,ZH,Affoltern,Dietikon–Schlieren,Region Zürich
2,Affoltern am Albis,ZH,Affoltern,Dietikon–Schlieren,Region Zürich
3,Bonstetten,ZH,Affoltern,Horgen–Wädenswil,Region Zürich
4,Hausen am Albis,ZH,Affoltern,Zug,Zentralschweiz
5,Hedingen,ZH,Affoltern,Dietikon–Schlieren,Region Zürich


# Add region info to customers

In [22]:
swiss_customer = ek_info.EK_Land == "SCHWEIZ"

### Find a GDENR for each swiss customer

#### First, match by PLZ

In [23]:
ek_info["GDENR_PLZ"] = plz_unique.reindex(
    pd.to_numeric(ek_info.EK_Plz, errors="coerce").fillna(-1).astype("int64")
).GDENR.values

#### Then, match by full town name

In [24]:
gdenr_by_name = plz.groupby('NAME').agg({'GDENR': 'first'})

ek_info["GDENR_NAME"] = (
    gdenr_by_name.reindex(ek_info.EK_Ort).values
)

#### Then, match by first word of town name (for, e.g. *Laufen (BL)*)

In [25]:
def firstnames(s):
    s = s.fillna("")
    return s.str.partition(expand=False).apply(lambda x: x[0])


gdenr_by_firstname = (
    plz.assign(FIRSTNAME=firstnames(plz.NAME))
    .groupby("FIRSTNAME")
    .agg({"GDENR": "first"})
)

ek_info["GDENR_FIRSTNAME"] = gdenr_by_firstname.reindex(
    firstnames(ek_info.EK_Ort)
).values

#### Finally, match some special cases by remapping them to the closest town, then match by full name

In [26]:
ek_info["GDENR_SPECIAL"] = gdenr_by_name.reindex(
    ek_info.EK_Ort.replace(
        {
            "Schönbühl Einkaufszentrum": "Urtenen-Schönbühl",
            "Emmenbrücke 1": "Emmen",
            "Glattzentrum b. Wallisellen": "Wallisellen",
            "Zürich-Flughafen": "Kloten",
            "Büsingen": "Schaffhausen",
            "Serfontana": "Chiasso",
            "Triesen": "Sevelen",
            "Campione d'Italia": "Bissone",
        }
    )
).values

#### Merge all matches by successive fallback

In [27]:
ek_info["GDENR"] = (
    ek_info.GDENR_PLZ
    .fillna(ek_info.GDENR_NAME)
    .fillna(ek_info.GDENR_FIRSTNAME)
    .fillna(ek_info.GDENR_SPECIAL)
)

# Delete matches of non-swiss customers (e.g. by PLZ or first name)
ek_info.loc[~swiss_customer, 'GDENR'] = None

ek_info.drop(
    "GDENR_PLZ GDENR_NAME GDENR_FIRSTNAME GDENR_SPECIAL".split(), axis="columns", inplace=True
)

#### Sanity check: this should be empty!

In [28]:
not_matched = ek_info.loc[
    ek_info.GDENR.isnull()
    & swiss_customer,
    ["EK_Plz", "EK_Ort"],
]

not_matched

,EK_Plz,EK_Ort


### Join region info on GDENR

In [29]:
ek_region = (
    ek_info[["Endkunde_NR", "GDENR"]]
    .merge(reg, on="GDENR", how="left")
    .drop("GDENR", axis="columns")
    .reset_index(drop=True)
)

ek_region.head(20)

,Endkunde_NR,NAME,KANTON,BEZIRK,REGION,GROSSREGION
0,100034,Winterthur,ZH,Winterthur,Winterthur,Region Zürich
1,100039,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
2,100061,Winterthur,ZH,Winterthur,Winterthur,Region Zürich
3,100064,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
4,100066,Uster,ZH,Uster,Uster–Dübendorf,Region Zürich
5,100083,Volketswil,ZH,Uster,Uster–Dübendorf,Region Zürich
6,100092,Zürich,ZH,Zürich,Zürich,Region Zürich
7,100095,Wetzikon (ZH),ZH,Hinwil,Wetzikon (ZH),Region Zürich
8,100097,Dübendorf,ZH,Uster,Uster–Dübendorf,Region Zürich
9,100098,Wallisellen,ZH,Bülach,Uster–Dübendorf,Region Zürich


#### Sanity check: this should be empty!

In [30]:
ek_info.loc[~swiss_customer & ek_region.KANTON.notnull()]

,Endkunde_NR,Endkunde,EK_Aktiv,EK_Land,EK_Plz,EK_Ort,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Kamp_Erfass_Jahr_min,Kamp_Erfass_Jahr_max,GDENR


# Write out result

In [31]:
store_bin(ek_region, 'ek_region.feather')

2019-09-04 11:35:54 [INFO] Writing to file C:\Users\kpf\data\Raumgliederung\ek_region.feather
2019-09-04 11:35:54 [INFO] Written 2.1 MB
2019-09-04 11:35:54 [INFO] Finished storing binary file in 0.02s (0.02s CPU)
